In [13]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [61]:
participant_metadata = read.table("~/sc-online/notebook_data/pd/calico_donor_metadata.tsv", sep="\t", header=T)
participant_metadata$case_control = ifelse(
    participant_metadata$disease == "Control", "ctr", "pd"
)
head(participant_metadata)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”


,participant_id,age,country,sex,disease,case_control
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,1995-078,80,Netherlands,Female,Control,ctr
2,1997-115,89,Netherlands,Female,Control,ctr
3,1998-016,82,Netherlands,Female,Control,ctr
4,1998-051,94,Netherlands,Female,Control,ctr
5,2002-086,74,Netherlands,Female,Control,ctr
6,2007-020,74,Netherlands,Male,Control,ctr


In [62]:
manifest_orig = read.table("~/sc-online/notebook_data/pd/Calico_Macosko_PD_GSA_Terra_Manifest_20240220.tsv", sep="\t", header=TRUE)
manifest_orig = manifest_orig[complete.cases(manifest_orig),]
manifest_orig$age = manifest_orig$Age 
manifest_orig$sex = manifest_orig$Sex
manifest_orig$case_control = ifelse(
    manifest_orig$Disease == "Control", "ctr", "pd"
)
manifest_orig$participant_id = manifest_orig$collaborator_participant_id


In [63]:
repatient = read.table("~/sc-online/notebook_data/pd/calico_repatient_20240304.tsv", sep="\t", header=FALSE)
repatient$chip_well_barcode = repatient$V2
repatient$old_participant_id = repatient$V5
repatient$participant_id = repatient$V6

In [64]:
manifest = (
    manifest_orig 
    %>% left_join(repatient, by="chip_well_barcode", suffix=c("_orig", "_repatient"))
    %>% mutate(
        participant_id = ifelse(
            is.na(participant_id_repatient), participant_id_orig, participant_id_repatient
        )
    )
)

In [65]:
head(manifest[,c("participant_id", "chip_well_barcode", "age", "sex", "case_control")])

,participant_id,chip_well_barcode,age,sex,case_control
,<chr>,<chr>,<int>,<chr>,<chr>
1,PDC091,206954930010_R05C02,85,Male,ctr
2,PDC110,206954930060_R11C02,94,Female,ctr
3,PDC078,206954930093_R05C02,91,Male,ctr
4,PDC078,207762950055_R07C02,91,Male,ctr
5,2019-102,207762950055_R11C01,58,Male,pd
6,PD0878,207762950086_R02C01,71,Male,pd


In [68]:
# write manifest to a tsv 
write.table(manifest, "~/sc-online/notebook_data/pd/calico_donor_corrected_manifest_20240306.tsv", sep="\t", quote=FALSE, row.names=FALSE)